In [ ]:
print("Hello World")

Hello World


In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-02-23 23:47:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Connecting to 127.0.0.1:7897... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.13MB/s    in 0.3s    

2026-02-23 23:47:42 (3.13 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
len(text)

1115394